# Распознавание спама в СМС-сообщениях
_Контест 1_

Сам контест и данные — [тут](https://www.kaggle.com/c/competition-1-lingvo-hse-2018-spam-detection).

## Загружаем данные

Содержание файлов:
* `texts.csv` - текстовое содержание СМС сообщений;
* `train.csv` - идентификаторы СМС сообщений и их метки класса (обучающая выборка);
* `test.csv` - идентификаторы СМС сообщений (тестовая выборка);
* `sample_submission.csv` - пример корректного submission

Описание колонок:
* `ID` - идентификатор СМС сообщения;
* `TEXT` - текстовое содержание СМС сообщения;
* `SPAM_PROB` - вероятность того, что сообщение является спамом;
* `CATEGORY` - метка класса (spam - сообщение со спамом)

In [1]:
import pandas as pd
import numpy as np

## Данные

In [2]:
texts = pd.read_csv("./texts.csv")
train = pd.read_csv("./train.csv")
test_ids = pd.read_csv("./test.csv")["ID"].as_matrix()

### Решаем проблемы

#### Бинарные метки классов
В `train.csv` классы обозначены как `ham` и `spam`. Поменяем их так, чтобы `ham` стало равно 0, а `spam` — 1.

In [3]:
def make_binary(data):
    if data == "ham":
        data = 0
    else:
        data = 1
    return data

train["CATEGORY"] = train["CATEGORY"].apply(make_binary)

#### Отсеиваем лишнее
Чтобы нам не мешались тестовые данные, сразу отсеем сообщения с `ID`, попавшими в тест.

In [4]:
test_texts = texts.loc[texts["ID"].isin(test_ids)]
test_texts.head()

,ID,TEXT
3,3,Do you like Italian food?
10,10,New TEXTBUDDY Chat 2 horny guys in ur area 4 j...
13,13,Hey now am free you can call me.
15,15,U having lunch alone? I now so bored...
17,17,Todays Voda numbers ending with 7634 are selec...


#### Выделяем тексты и целевую переменную отдельно

In [5]:
df = texts.merge(train).drop("ID", axis=1)
df.head()

,TEXT,CATEGORY
0,Let me know how to contact you. I've you settl...,0
1,Where can download clear movies. Dvd copies.,0
2,Desires- u going to doctor 4 liver. And get a ...,0
3,"House-Maid is the murderer, coz the man was mu...",0
4,True. Its easier with her here.,0


In [6]:
texts_train = df["TEXT"].as_matrix()
y = df["CATEGORY"].as_matrix()

## Фичи

### TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
vectorizer = TfidfVectorizer(stop_words="english")
corpus = texts["TEXT"].as_matrix()
vectorizer = vectorizer.fit(corpus)

In [9]:
X = vectorizer.transform(texts_train)

## Ура, машинное обучение

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
def grid_and_count(model, model_str, param_grid):
    model = GridSearchCV(model, param_grid)
    model.fit(X_train, y_train)
    
    y_out = model.predict(X_test)
    performance = roc_auc_score(y_test, y_out)
    print("Модель: {}\nПараметры: {}\nROC-AUC = {}".format(model_str, model.best_params_, performance))
    return model

### Decision Tree

In [13]:
from sklearn.tree import DecisionTreeClassifier

In [14]:
param_grid = {"max_depth": np.arange(3, 100)}
tree = DecisionTreeClassifier()
tree = grid_and_count(tree, "DecisionTree", param_grid)

Модель: DecisionTree
Параметры: {'max_depth': 74}
ROC-AUC = 0.902735181632678


### Random forest

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
param_grid = {"n_estimators": np.arange(2,100)}
forest = RandomForestClassifier()
forest = grid_and_count(forest, "RandomForest", param_grid)

Модель: RandomForest
Параметры: {'n_estimators': 41}
ROC-AUC = 0.9015438472159432


### Naïve Bayes

Используем Multinomial Naïve Bayes, потому что у нас векторы (значит, не Gaussian) с небинарными значениями (значит, не Bernoulli).

In [17]:
from sklearn.naive_bayes import MultinomialNB

In [18]:
multi_bayes = MultinomialNB()
multi_bayes.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
y_out = multi_bayes.predict(X_test)
multi_performance = roc_auc_score(y_test, y_out)
print("Успех по метрике ROC-AUC = {}".format(multi_performance))

Успех по метрике ROC-AUC = 0.8865299788614508


## Считаем для тестовых данных

In [23]:
X_goal = vectorizer.transform(test_texts["TEXT"].as_matrix())

In [24]:
def count_predictions(model, model_str):
    predictions = model.predict_proba(X_goal)[:, 1]
    result_dict = {"ID": test_ids, "SPAM_PROB": predictions}
    result_df = pd.DataFrame(result_dict)
    result_df.to_csv("./submission_{}.csv".format(model_str), sep=",", encoding="utf-8", index=None)

In [25]:
count_predictions(tree, "decisionTree")
count_predictions(forest, "randomForest")
count_predictions(multi_bayes, "multinomialBayes")